### **Exame de CT-213 (Inteligência Artificial para Robótica Móvel)**
### **Alunos:** Marcelo Roncato Júnior, Rodrigo Jamundá Melo, Thiago Zanfolin
### **Grupo:** 16

# **Implementação de Rede Neural Convolucional para Classificação de Gêneros Musicais**
### **Professor:** Marcos Ricardo Omena de Albuquerque Máximo

**Instituto Tecnológico de Aeronáutica – ITA**

# 1. Introdução

Este notebook corresponde ao código-fonte do projeto "Implementação de Rede Neural Convolucional para Classificação de Gêneros Musicais", para o exame de CT-213 do grupo 16. O código foi feito no *Google Colaboratory* pela facilidade de implementação e didática de explicação.

Foram desenvolvidas 2 arquiteturas de rede neural (*neural network*, NN): uma convolucional (*convolutional neural network*, CNN) e outra que une características de CNN e RNN (*residual neural networks*). Como *dataset*, foi utilizada a versão *small* do *Free Music Archive* (FMA-Small). FMA (https://freemusicarchive.org/) é uma plataforma que contém músicas de licença aberta produzidas por artistas independentes. A versão *small* do *dataset* (aproximadamente 7,2 GB) contém 8000 faixas de 30 segundos cada distribuídas em 8 gêneros (*Electronic*, *Experimental*, *Folk*, *Internacional*, *Instrumental*, *Hip-Hop*, *Rock* e *Pop*).

Acima de cada célula, está a estimativa do tempo para ela rodar, considerando a utilização da GPU T4.

# 2. Carregamento do *dataset*

A célula seguinte baixa e descompacta o *dataset*.

Estimativa de tempo: 16 min (6 min para baixar e 10 min para descompactar).

In [ ]:
import zipfile
import os

# Baixar o dataset FMA small
!wget https://os.unil.cloud.switch.ch/fma/fma_small.zip

# Descompactar o dataset
with zipfile.ZipFile("fma_small.zip", "r") as zip_ref:
    zip_ref.extractall("fma_small")

# Remover arquivo zip para economizar espaço
os.remove("fma_small.zip")

Caso esteja dando erro na descompactação (se rodar muitas vezes a célula acima consecutivamente, isso pode acontecer), rodar a célula abaixo.

In [ ]:
if os.path.exists("fma_small.zip"): # Excluir arquivo caso esteja dando erro na descompactação do FMA-Small
    os.remove("fma_small.zip")

A célula seguinte baixa e descompacta os metadados.

Estimativa de tempo: 1 min 30 s.

In [ ]:
# Baixar os metadados
!wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip

# Descompactar os metadados
with zipfile.ZipFile("fma_metadata.zip", "r") as zip_ref:
    zip_ref.extractall("fma_metadata")

# Remover arquivo zip para economizar espaço
os.remove("fma_metadata.zip")

Os metadados fazem o mapeamento e descrição do *dataset* (ID da faixa, gênero, artista, álbum, licença, data de lançamento, características como timbre e BPM, etc.). Caso queira visualizar os arquivos para entender melhor, rode a próxima célula.

Estimativa de tempo: instantâneo.

In [ ]:
# Listar os arquivos e subpastas da pasta fma_metadata
for root, dirs, files in os.walk("fma_metadata"):
    for name in files:
        print(os.path.join(root, name))

A próxima célula atribui à variável *tracks* o carregamento do arquivo CSV tracks.csv dos metadados em um *dataframe* Multiindex (estrutura de dados do Pandas com múltiplos níveis de indexação). Nesse caso, o *dataframe* possui 2 camadas de colunas.

Estimativa de tempo: 4 s.

In [4]:
import pandas as pd

# tracks carrega o arquivo CSV tracks.csv em um dataframe Multiindex com 2 camadas de colunas
# A coluna 0 contém os IDs das faixas
tracks = pd.read_csv('fma_metadata/fma_metadata/tracks.csv', header=[0, 1], index_col=0)

A célula seguinte testa algumas faixas aleatórias. Pode rodar várias vezes, caso queira ouvir diferentes gêneros.

Estimativa de tempo: instantâneo.

In [ ]:
from IPython.display import Audio
import random

# Carregar os IDs das faixas do FMA-small
track_ids = tracks.loc[tracks[('set', 'subset')] == 'small'].index # Filtra os dados do subset 'small' do dataset, dentro da categoria 'set'. Assim,
                                                                   # track_ids contém todos os IDs de faixas que estão no conjunto FMA-small

# Mapeia ID da faixa <--> gênero musical principal de cada faixa
track_genres = tracks.loc[track_ids][('track', 'genre_top')]

# Pegar 3 IDs aleatórios
sample_ids = random.sample(list(track_ids), 3)
print("IDs de faixas escolhidas:", sample_ids)

for track_id in sample_ids:
    folder = f"{track_id:06d}"[:3]  # Pega os 3 primeiros dígitos para navegar na pasta correta
    filepath = f"fma_small/fma_small/{folder}/{track_id:06d}.mp3"

    print(f"\nTocando faixa ID {track_id} — Gênero: {track_genres[track_id]}")
    display(Audio(filename=filepath))

# 3. Construção do 1º modelo (CNN)

|**Camada**|**Tipo**|**Número de Filtros**|**Tamanho da Saída**|**Tamanho do Kernel**|**Stride**|**Função de Ativação**|
|:------|:----------------|:--|:----|:---|:-|:------|
|Entrada|Imagem (espectrograma)|-  |40x1024|-   |- |-      |
|1      |Conv2D           |32  |40x1024x32|3x3 |1 |LeakyReLU   |
|2      |Conv2D |64  |40x1024x64|5x5 |1 |LeakyReLU    |
|3      |BatchNormalization           |- |40x1024x64|- |- |-   |
|4      |AveragePooling2D |64 |20x512x64  |2x2 |2 |-      |
|5      |Conv2D           |64|20x512x64  |3x3 |1 |LeakyReLU   |
|6      |Conv2D     |128  |20x512x128   |5x5   |1 |LeakyReLU   |
|7      |BatchNormalization    |-  |20x512x128   |-   |- |- |
|8       |AveragePooling2D      |128 |10x256x128 |2x2 |2 |- |
|9      |GlobalAveragePooling  |128  |128  |- |- |-|
|10      |Dense (FC)  |- |50 |- |- |ReLU |
|11     |Dense (FC)  |- |5 |- |- |softmax |

<p align="center">
<b>Tabela 1</b>: arquitetura do modelo CNN. </p>

Acurácia obtida no teste após treinamento: 0.65

*Loss* obtida no teste após treinamento: 0.99

Estimativa de tempo: 54

In [6]:
from tensorflow.keras import layers, activations, Input, regularizers
from tensorflow.keras.models import Sequential

lambda_l2 = 0.016 # Parâmetros para regularização L2

# Para facilitar entendimento dos comentários acima de cada camada da rede:
# f: dimensões do filtro
# s: dimensões de stride
# nc': número de filtros

def make_CNN_model(input_shape, output_dim):
    model = Sequential()

    model.add(Input(shape=input_shape))

    # 1ª camada: Conv2D, nc' = 32, f = 3, s = 1, LeakyReLU
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='linear'))
    model.add(layers.LeakyReLU(alpha=0.01))

    # 2ª camada: Conv2D, nc' = 64, f = 5, s = 1, LeakyReLU
    model.add(layers.Conv2D(filters=64, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='linear'))
    model.add(layers.LeakyReLU(alpha=0.01))

    # 3ª camada: BatchNormalization
    model.add(layers.BatchNormalization()) #

    # 4ª camada: AveragePooling2D, f = 2, s = 2
    model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))

    # 5ª camada: Conv2D, nc' = 64, f = 3, s = 1, LeakyReLU
    model.add(layers.Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same', activation='linear'))
    model.add(layers.LeakyReLU(alpha=0.01))

    # 6ª camada: Conv2D, nc' = 128, f = 5, s = 1, LeakyReLU, regularização L2
    model.add(layers.Conv2D(filters=128, kernel_size=(5, 5), strides=(1, 1), padding='same', activation='linear', kernel_regularizer=regularizers.l2(lambda_l2)))
    model.add(layers.LeakyReLU(alpha=0.01))

    # 7ª camada: BatchNormalization
    model.add(layers.BatchNormalization())

    # 8ª camada: AveragePooling2D, f = 2, s = 2
    model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2)))

    # 9ª camada: GlobalAveragePooling2D
    model.add(layers.GlobalAveragePooling2D())

    # 10ª camada: Fully-Conected, 50 neurônios, ReLU, regularização L2
    model.add(layers.Dense(50, activation='relu', kernel_regularizer=regularizers.l2(lambda_l2)))
    model.add(layers.Dropout(0.5)) # Adicionar dropout, para reduzir overfitting

    # 11ª camada: Fully-Conected, softmax
    model.add(layers.Dense(output_dim, activation=activations.softmax))

    return model


# 4. Construção do 2º modelo (CNN + RNN)

In [ ]:
from tensorflow.keras import layers, activations, Input
from tensorflow.keras.models import Sequential


def make_clash_model(input_shape, output_dim, kernel_dim=5, kernel_initial_num=32, rnn_cells=128, dropout=0.25):
    model = Sequential()

    model.add(Input(shape=input_shape))



Exemplo "completo" gerado pelo GPT:

# 5. Definição de funções úteis para processamento dos dados e treinamento

Estimativa de tempo: instantâneo.

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
import librosa
import librosa.display
from sklearn.model_selection import train_test_split

FMA_DIR = '/content/fma_small/fma_small' # Caminho onde estão os arquivos .mp3 do FMA-small
METADATA_PATH = '/content/fma_metadata/fma_metadata/tracks.csv' # Caminho para o arquivo tracks.csv
TRAINING_DATA_DIR = '/content/training_data' # Caminho para os dados de treinamento
TARGET_GENRES = ['Rock', 'Hip-Hop', 'Experimental', 'Folk', 'Pop']  # Gêneros de interesse
SAMPLES_PER_GENRE = 1000  # Número de exemplos por gênero para balancear o dataset
SPECTROGRAM_SHAPE = (40, 1024) # Tamanho fixo para o espectrogrmaa

def get_metadata(path=METADATA_PATH):
    """
    Lê o tracks.csv com dois níveis de cabeçalho (header=[0, 1])
    (o arquivo tem colunas multi-indexadas)
    Usa a coluna de índice (index_col=0) para que cada linha represente
    uma faixa (track) identificada por seu ID numérico.

    :param path: caminho até o arquivo tracks.csv
    :type path: string
    :return tracks: "tabela" com os metadados
    :rtype: pd.MultiIndex
    """
    tracks = pd.read_csv(METADATA_PATH, header=[0, 1], index_col=0)

    return tracks

def filter_tracks(tracks, genres=TARGET_GENRES):
    """
    Filtra as faixas a partir da "tabela" de dados e do seu gênero

    :param tracks: "tabela" com as faixas
    :type tracks: pd.MultiIndex
    :param genres: gêneros de interesse
    :type genres: list
    :return filtered_tracks: "tabela" de dados filtrada
    :rtype: pd.MultiIndex
    """
    # Filtra apenas as faixas do subset "small"
    small_tracks = tracks[tracks[('set', 'subset')] == 'small']

    # Remove faixas que não possuem um gênero principal definido (campo 'genre_top' vazio)
    small_tracks = small_tracks.dropna(subset=[('track', 'genre_top')])

    # Filtra apenas as faixas que pertencem aos gêneros definidos
    filtered_tracks = small_tracks[small_tracks[('track', 'genre_top')].isin(TARGET_GENRES)]

    return filtered_tracks

def balance_genres(tracks, genres=TARGET_GENRES, max_samples=SAMPLES_PER_GENRE):
    """
    Balanceia o número de faixas por gênero

    :param tracks: "tabela" com as faixas (base de dados)
    :type tracks: pd.MultiIndex
    :param genres: gêneros de interesse
    :type genres: list
    :param max_samples: número máximo de faixas por gênero
    :type max_samples: int
    :return balanced_tracks: "tabela" de dados balanceada
    :rtype: pd.MultiIndex
    """
    balanced_tracks = []

    # Filtra por um gênero de cada vez
    for genre in genres:
        genre_tracks = tracks[tracks[('track', 'genre_top')] == genre]

        # Limita o tamanho de exemplos por gênero
        num_samples = min(max_samples, len(genre_tracks))

        # Sorteia aleatoriamente um número fixo de faixas para balancear o dataset
        balanced_tracks.append(genre_tracks.sample(num_samples, random_state=42))

    # Une os subconjuntos em um único dataframe
    balanced_tracks = pd.concat(balanced_tracks)

    return balanced_tracks

def get_audio_path(track_id, base_dir=FMA_DIR):
    """
    Converte o ID da faixa para o caminho do arquivo na base de dados

    :param track_id: ID da faixa
    :type track_id: int
    :param base_dir: diretório base do dataset
    :type base_dir: string
    :return track_path: caminho do arquivo na base de dados
    :rtype track_path: string
    """
    id_formatted = f"{track_id:06d}"
    track_path = os.path.join(base_dir, id_formatted[:3], f"{id_formatted}.mp3")

    return track_path

def audio_to_mel_spectrogram(file_path, duration=30, sr=22050, n_mels=SPECTROGRAM_SHAPE[0]):
    """
    Converte a faixa em um espectrograma

    :param file_path: arquivo da faixa
    :type file_path: string
    :param duration: duração da faixa em segundos
    :type duration: int
    :param sr: taxa de amostragem
    :type sr: int
    :n_mels: número de bandas
    :type n_mels: int
    :return mel_db: espectrograma em escala logarítimica processado
    :rtype mel_db: pd.DataFrame
    """
    try:
        y, sr = librosa.load(file_path, duration=duration, sr=sr)
        mel = librosa.feature.melspectrogram(y=y, sr=SPECTROGRAM_SHAPE[1]//duration, n_mels=n_mels)
        mel_db = librosa.power_to_db(mel, ref=np.max)
        return mel_db

    # Captura um erro em algum arquivo
    except Exception as e:
        print(f"Erro em {file_path}: {e}")
        return None


def process_training_data(genres):
    """
    Processa os dados das faixas para treinamento

    :param genres: gêneros de interesse
    :type genres: list
    :return X_train: input de treinamento
    :rtype X_train: numpay array
    :return X_test: input de avaliação
    :rtype X_test: numpay array
    :return y_train: output de treinamento
    :rtype y_train: numpay array
    :return y_test: output de avaliação
    :rype y_test: numpay array
    """
    # Listas para armazenar espectrogramas (X) e rótulos numéricos (y)
    X = []
    y = []

    # Processa e filtra as faixas de áudio
    tracks = get_metadata()
    filtered_tracks = filter_tracks(tracks)
    balanced_tracks = balance_genres(filtered_tracks)

    # Dicionário para mapear cada gênero para um índice (ex: {'Rock': 0, 'Hip-Hop': 1, ...})
    label_dict = {genre: idx for idx, genre in enumerate(genres)}

    # Itera sobre cada faixa
    for track_id, row in tqdm(balanced_tracks.iterrows(), total=balanced_tracks.shape[0]):
        # Define o gênero da faixa
        genre = row[('track', 'genre_top')]

        # Converte o caminho do arquivo
        file_path = get_audio_path(track_id)

        # Extrai o espectrograma
        spec = audio_to_mel_spectrogram(file_path)

        # Pula a faixa se houver erro no processamento
        if spec is None:
            continue

        # Ajusta o espectrograma para o tamanho correto
        if spec.shape[1] < SPECTROGRAM_SHAPE[1]:
            # Aplica padding se o espectrograma for menor que o esperado
            padding_width = SPECTROGRAM_SHAPE[1] - spec.shape[1]
            spec = np.pad(spec, ((0, 0), (0, padding_width)), mode='constant', constant_values=0)

        elif spec.shape[1] > SPECTROGRAM_SHAPE[1]:
            # Trunca se ele for maior
            spec = spec[:, :SPECTROGRAM_SHAPE[1]]

        # Normaliza os valores para [0, 1] (boa prática para CNNs)
        spec = (spec - spec.min()) / (spec.max() - spec.min())

        # Adiciona uma dimensão ao final (input shape (H, W, 1))
        spec = spec[..., np.newaxis]

        # Adiciona o espectrograma e o gênero aos dados de treinamento
        X.append(spec)
        y.append(label_dict[genre])

    # Transforma a lista de espectrogramas em um numpy array
    X = np.array(X)

    # Transforma os rótulos (ex: [0, 1, 2]) em formato one-hot
    y = to_categorical(np.array(y), num_classes=len(genres))

    # Divide em 80% treino / 20% teste, com random_state fixo para reprodutibilidade
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

def save_training_data(**data):
    """
    Salva os dados de treinamento em um arquivo .npz

    :param **data: dados de treinamento
    :type **data: dict
    :return void:
    """
    # Cria o diretório caso ele não exista
    if not os.path.exists(TRAINING_DATA_DIR):
        os.makedirs(TRAINING_DATA_DIR)

    # Salva o arquivo com os dados
    data_filename = f"data_{SPECTROGRAM_SHAPE[0]}_{SPECTROGRAM_SHAPE[1]}_{SAMPLES_PER_GENRE}.npz"
    np.savez(os.path.join(TRAINING_DATA_DIR, data_filename), **data)

def get_training_data(genres=TARGET_GENRES):
    """
    Carrega os dados de treinamento

    :param genres: gêneros de interesse
    :type genres: list
    :return X_train: input de treinamento
    :rtype X_train: numpy array
    :return X_test: input de avaliação
    :rtype X_test: numpy array
    :return y_train: output de treinamento
    :rtype y_train: numpy array
    :return y_test: output de avaliação
    :rtype y_test: numpy array
    """
    # Busca e abre o arquivo com os dados
    try:
        data_filename = f"data_{SPECTROGRAM_SHAPE[0]}_{SPECTROGRAM_SHAPE[1]}_{SAMPLES_PER_GENRE}.npz"
        data = np.load(os.path.join(TRAINING_DATA_DIR, data_filename))

        return data['X_train'], data['X_test'], data['y_train'], data['y_test']

    # Se o arquivo não existir, os dados são processados e o arquivo é criado
    except FileNotFoundError:
        X_train, X_test, y_train, y_test = process_training_data(genres)
        save_training_data(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test)

        return X_train, X_test, y_train, y_test

def delete_training_data():
    """
    Deleta os dados de treinamento

    :return void:
    """
    # Checa se o diretório existe
    if not os.path.exists(TRAINING_DATA_DIR):
        return

    # Abre o diretório e deleta tudo
    for filename in os.listdir(TRAINING_DATA_DIR):
        filepath = os.path.join(TRAINING_DATA_DIR, filename)
        if os.path.isfile(filepath):
            os.remove(filepath)

# 6. Treinamento e avaliação dos modelos CNN e CNN + RNN

Caso queira deletar os dados de algum treinamento anterior:

In [8]:
delete_training_data()

Execução do treinamento (comente/descomente os trechos indicados para alternar entre os 2 modelos de rede neural):

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import load_model

NUM_EPOCHS = 90
BATCH_SIZE = 65

def train_model(model, X_train, y_train, epochs, batch_size,
                optimizer=tf.keras.optimizers.Adam,
                loss_function=tf.keras.losses.categorical_crossentropy,
                metrics=['accuracy'],
                validation_split=0.2,
                is_verbose=False):
    """
    Realiza o treinamento do modelo

    :param model: modelo a ser treinado
    :type model: tf.keras.model
    :param X_train: input de treinamento
    :type X_traind: np.array
    :param y_train: output de treinamento
    :type y_train: np.array
    :param epochs: número de épocas de treinamento
    :type epochs: int
    :param batch_size: tamanho do batch
    :type batch_size: int
    :param optimizer: otimizador de compilação do modelo
    :type optimizer: tf.keras.optimizers
    :param loss_function: função de custo do modelo
    :type loss_function: tf.keras.losses
    :param metrics: métricas avaliadas
    :type metrics: list
    :param validation_split: fração do dataset de treinamento usado para validação
    :type validation_split: float
    :param is_verbose: controla se o treinamento é impresso na tela
    :type is_verbose: bool
    :return model: modelo treinado
    :rtype model: tf.keras.model
    :return history: histórico de treinamento
    :rtype history: tf.keras.callbacks.History
    """

    # Durante o treinamento, foi percebido que fazer um schedule da taxa de aprendizado melhorava o desempenho das redes
    # Por tentativa e erro, chegou-se no fator 0.35 (OBS.: esse número não foi otimizado, por falta de tempo de processamento)
    reduce_lr = ReduceLROnPlateau(
      monitor='val_loss',  # Métrica a ser monitorada
      factor=0.35,         # Fator pelo qual a taxa de aprendizado (learning rate) será reduzida (nova_lr = lr * factor)
      patience=5,          # Número de épocas sem melhoria após as quais a taxa de aprendizado será reduzida
      min_lr=0.0001,       # Taxa de aprendizado mínima
      mode='min',          # 'min' para val_loss (busca minimizar)
      verbose=1            # 1 para imprimir mensagens quando a taxa de aprendizado for atualizada
    )

    model.compile(optimizer=optimizer(),
              loss=loss_function,
              metrics=metrics)

    history = model.fit(X_train, y_train,
                    validation_split=validation_split,
                    epochs=epochs,
                    batch_size=batch_size, callbacks=[reduce_lr],
                    verbose=is_verbose)

    return model, history

def evaluate_model(model, X_test, y_test):
    """
    Avalia o treinamento do modelo

    :param model: modelo a ser avaliado
    :type model: tf.keras.model
    :param X_test: input de avaliação
    :type X_test: numpy array
    :param y_test: output de avaliação
    :type y_test: numpy array
    :return test_loss: função de custo da avaliação
    :rtype test_loss: float
    :return test_acc: acurácia da avaliação
    :rtyoe test_acc: float
    """

    test_loss, test_acc = model.evaluate(X_test, y_test)
    return test_loss, test_acc

# -------------------------
#   TREINAMENTO DAS REDES
# -------------------------

X_train, X_test, y_train, y_test = get_training_data()

# Para treinar o 1º modelo de rede, deixe as 4 linhas abaixo descomentadas:
model_CNN = make_CNN_model(SPECTROGRAM_SHAPE + (1,), len(TARGET_GENRES))
model_CNN.summary()
model_CNN, history = train_model(model_CNN, X_train, y_train, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, is_verbose=True)
test_loss, test_acc = evaluate_model(model_CNN, X_test, y_test)
model.save("model_CNN.h5")

# Para treinar o 2º modelo de rede, comente as 4 linhas acima e descomente as 4 linhas abaixo:
# model_RNN_CNN = make_RNN_CNN_model(SPECTROGRAM_SHAPE + (1,), len(TARGET_GENRES))
# model_RNN_CNN.summary()
# model_RNN_CNN, history = train_model(model_RNN_CNN, X_train, y_train, epochs=NUM_EPOCHS, batch_size=BATCH_SIZE, is_verbose=True)
# test_loss, test_acc = evaluate_model(model_RNN_CNN, X_test, y_test)
# model.save("model_CNN_RNN.h5")

print(f"Acurácia no teste: {test_acc:.2f}")

# Gráfico de acurácia
plt.figure()
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title('Acurácia durante o treinamento')
plt.xlabel('Épocas')
plt.ylabel('Acurácia')
plt.legend()

print(f"Loss no teste: {test_loss:.2f}")

# Gráfico de loss
plt.figure()
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title('Loss durante o treinamento')
plt.xlabel('Épocas')
plt.ylabel('Loss')
plt.legend()

plt.show()

Matriz de confusão:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Escolher modelo para avaliação
model = load_model("model_CNN.h5")
# model = load_model("model_CNN_RNN.h5")

# Previsões
y_pred = model.predict(X_test)
y_pred_labels = y_pred.argmax(axis=1)
y_true_labels = y_test.argmax(axis=1)

# Matriz
cm = confusion_matrix(y_true_labels, y_pred_labels)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=TARGET_GENRES)
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title("Matriz de Confusão")
plt.show()

Curvas ROC por classe:

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

# Binariza y_test se ainda não estiver
y_true = y_test
n_classes = y_test.shape[1]

plt.figure(figsize=(10, 8))

for i in range(n_classes):
    fpr, tpr, _ = roc_curve(y_true[:, i], y_pred[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"{TARGET_GENRES[i]} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], "k--", label="Random")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Curvas ROC por Gênero")
plt.legend()
plt.grid()
plt.show()

Curvas de precisão e revocação:

In [ ]:
from sklearn.metrics import classification_report

print("Relatório de classificação:\n")
print(classification_report(y_true_labels, y_pred_labels, target_names=TARGET_GENRES))